In [3]:
!pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 49.5 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [google-generativeai] 16/17 [google-generativeai]e]nt]


In [6]:
import google.generativeai as genai

In [10]:
genai.configure(api_key="")

In [12]:
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content("Is it too late to join the course?")

In [16]:
response.candidates[0].content.parts[0].text

"Whether or not it's too late to join a course depends on several factors:\n\n*   **The start date:** When did the course begin? If it started very recently (within the last few days), you might still be able to join. If it started weeks or months ago, it's likely too late.\n\n*   **The course structure:** Is it self-paced, or does it have live lectures/discussions? Self-paced courses are often more flexible, and you might be able to catch up. Courses with live components might be harder to join late.\n\n*   **The enrollment policy:** Does the course have a strict enrollment deadline? Some courses might allow late enrollment with instructor permission.\n\n*   **Your ability to catch up:** Are you willing and able to dedicate extra time to catch up on missed material?\n\n*   **The course provider:** Contacting the course provider and asking if it's possible to still join is the best way to find out.\n\n**How to find out if you can still join:**\n\n1.  **Check the course website:** Look 

In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
!pip install requests

In [20]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
!pip install elasticsearch

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [26]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [27]:
query = "How do execute a command on a Kubernetes pod?"

In [38]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [40]:
print(max(elastic_search(query)))

44.50556


In [41]:
query2 = "How do copy a file to a Docker container?"

In [45]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query2,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [46]:
response = es_client.search(index=index_name, body=search_query)

In [48]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'I2TxepcB5oEp3ok-RSCK',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'QmTxepcB5oEp3ok-RSD5',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [58]:
def elastic_search2(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [59]:
result2 = elastic_search2(query2)[2]['question']

In [60]:
result2

'How do I copy files from a different folder into docker container’s working directory?'

In [63]:
search_results = elastic_search2(query2)

In [71]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    context = ""
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [72]:
prompt = build_prompt(query2, search_results)

In [74]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT:
    Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory conta

In [75]:
len(prompt)

1474

In [76]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]


In [78]:
import tiktoken

In [79]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [80]:
tokens = encoding.encode(prompt)

In [81]:
len(tokens)

327

In [82]:
def llm(prompt):
    model = genai.GenerativeModel("gemini-2.0-flash")

    response = model.generate_content([{"role": "user", "parts": [prompt]}])
    
    return response.candidates[0].content.parts[0].text.strip()

In [83]:
answer = llm(prompt)

In [84]:
print(answer)

You can copy files from your local machine into a Docker container using the `docker cp` command.
To copy a file or directory from your local machine into a running Docker container, you can use the following syntax:
`docker cp /path/to/local/file_or_directory container_id:/path/in/container`

In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
`COPY ["src/predict.py", "models/xgb_model.bin", "./"]`
